In [43]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html
# https://stackoverflow.com/questions/45429831/valueerror-a-value-in-x-new-is-above-the-interpolation-range-what-other-re

import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

In [29]:
power_draw = np.loadtxt("/home/lucas/Documents/perceptronac/results/exp_1658289067/exp_1658289067_power_draw.csv")

In [33]:
data = pd.read_csv("/home/lucas/Documents/perceptronac/results/exp_1658289067/exp_1658289067_valid_values.csv")

In [45]:
new_data = data[["topology","quantization_bits","start_time","end_time"]] #.set_index("topology")

In [72]:
x = power_draw[:,0]
y = power_draw[:,1]

In [73]:
# fig,ax = plt.subplots(nrows=1, ncols=1,figsize=(20,20))
# ax.plot(x, y, 'o')
# plt.show()

In [74]:
f = interpolate.interp1d(x, y,fill_value="extrapolate")

In [75]:
# xnew = np.linspace(np.min(new_data["start_time"].values),np.max(new_data["end_time"].values) , 10000)

# ynew = f(xnew)   # use interpolation function returned by `interp1d`

# fig,ax = plt.subplots(nrows=1, ncols=1,figsize=(20,20))
# ax.plot(xnew, ynew, '-')
# plt.show()

In [79]:
joules = (f((new_data["start_time"].values + new_data["end_time"].values)/2)-np.min(y)) * (new_data["end_time"].values - new_data["start_time"].values) 

In [83]:
data["joules"] = joules

In [106]:
plt.figure(figsize=(20,20))
plt.plot(data["(data_bits+model_bits)/data_samples"],data["joules"],linestyle="",marker="x")

In [101]:
from perceptronac.convex_hull import points_in_convex_hull

selected_points_mask,fig = points_in_convex_hull(data,"(data_bits+model_bits)/data_samples","joules",log_x=True)
plt.show()

print(data.set_index("topology").iloc[selected_points_mask,:])

In [102]:
data.iloc[np.argmin(data["(data_bits+model_bits)/data_samples"].values),:]

In [105]:
data.iloc[np.argmin(data["joules"].values),:]